In [111]:
from webexteamssdk import WebexTeamsAPI
from webexteamsbot import TeamsBot
import os
import requests
import re, datetime
from datetime import datetime, timedelta 
import pandas as pd
from webexteamssdk import WebexTeamsAPI
import requests
from openpyxl import Workbook
import openpyxl

In [ ]:
#In CMD

:: export TEAMS_BOT_URL=http://77af5e8d.ngrok.io/
:: export TEAMS_BOT_TOKEN=MTg0ZTdhOTMtOTE5NS00NzAxLTgwNWEtZDQwZmM1NmZiODk5ODJjODIxZmEtNmZm_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f
:: export TEAMS_BOT_EMAIL=timerec@webex.bot
:: export TEAMS_BOT_APP_NAME=Time Recording Bot

In [2]:
bot_email = os.getenv("TEAMS_BOT_EMAIL")
teams_token = os.getenv("TEAMS_BOT_TOKEN")
bot_url = os.getenv("TEAMS_BOT_URL")
bot_app_name = os.getenv("TEAMS_BOT_APP_NAME")

print(bot_app_name)

None


In [ ]:
################################################

In [90]:
bot_email = "timerec@webex.bot"
teams_token = "MTg0ZTdhOTMtOTE5NS00NzAxLTgwNWEtZDQwZmM1NmZiODk5ODJjODIxZmEtNmZm_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f"
bot_url = "https://8a909d6c.ngrok.io"
bot_app_name = "Time Recording Bot"
room_ID = "Y2lzY29zcGFyazovL3VzL0FQUExJQ0FUSU9OL2VhMDZiMTMyLWM5YzEtNDY1Zi05OGJmLTVmNzE3Y2RmYTMxYw"


headers = {'Authorization': 'Bearer ' + teams_token,
'Content-type': 'application/json;charset=utf-8'}
post_data = {'roomId': room_ID}
response = requests.post(bot_url, json=post_data, headers=headers)

api = WebexTeamsAPI(teams_token)

In [66]:
u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
r = requests.get(u).json()


returned_data = r["currentDateTime"].split("T")
cur_time = returned_data[1]
cur_time = (cur_time.split("+"))[0]
cur_time

'20:08'

In [131]:
bot = TeamsBot(
    bot_app_name,
    teams_bot_token=teams_token,
    teams_bot_url=bot_url,
    teams_bot_email=bot_email,
)

def greeting(incoming_msg):
    # Loopkup details about sender
    global sender
    sender = bot.teams.people.get(incoming_msg.personId)

    # Create a Response object and craft a reply in Markdown.
    return ("Hello {}, I'm your time recording bot. \n\n See what I can do by asking for **/help**.").format(sender.firstName)
    #return "See what I can do by asking for **/help**."
    
def record_time(incoming_msg):
    return date
    
def date(incoming_msg):
    global cur_date
    date_test = (incoming_msg.text).replace("/recordtime","")
    date_test = (date_test.replace(" ",""))
    if len(str(date_test)) != 8:
        if len(str(date_test)) > 0:
            reply = "Please enter a valid date format: /recording DDMMYYYY"
            return reply
            
        else:    
            global cur_date
            u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
            r = requests.get(u).json()

            if r["serviceResponse"]:
                return "Error: " + r["serviceResponse"]

            returned_data = r["currentDateTime"].split("T")
            cur_date = returned_data[0]
            reply = "Recording for {DATE}. \n\n Chose a start time by entering /start hr.".format(DATE=cur_date)
            return reply
    else: 
        cur_date = datetime.strptime(str(date_test), '%d%m%Y').date()
        reply = "Recording for {DATE}. \n\n Chose a start time by writing: /start hr.".format(DATE=cur_date)
        return reply
   
    
def start_time(incoming_msg):
    global start_time2
    start_time = (incoming_msg.text).replace("/start ","")
    start_time2 = datetime.strptime(str(start_time), '%H%M').time()
    #start_time = int(start_time)
    return ("Your start time for " + str(cur_date) + " is: {} \n\n Chose a start time by writing: /end hr.: ").format(start_time2)
            

def end_time(incoming_msg):
    global end_time_final
    end_time = (incoming_msg.text).replace("/end","")
    end_time = (end_time.replace(" ",""))
    if len(str(end_time)) != 3:
        if len(str(end_time)) != 4:
            if len(str(end_time)) > 0:
                reply = "Please enter a valid date format: /end HMM or HHMM"
                return reply
            else: 
                u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
                r = requests.get(u).json()

                returned_data = r["currentDateTime"].split("T")
                end_time = returned_data[1]
                end_time = (end_time.split("+"))[0]
                end_time_final = datetime.strptime(str(end_time), '%H%M').time()
                reply = "Your end time for " + str(cur_date) + " is: {TIME}.".format(TIME=end_time_final)
                return reply
        else:
            end_time_final = datetime.strptime(str(end_time), '%H%M').time()
            reply = "Your end time for " + str(cur_date) + " is: {TIME}.".format(TIME=end_time_final)
            return reply
    else:
        end_time_final = datetime.strptime(str(end_time), '%H%M').time()
        reply = "Your end time for " + str(cur_date) + " is: {TIME}. \n\n Save with: /save".format(TIME=end_time_final)
        return reply
    
def save(incoming_msg):
    global filename
    #sender = bot.teams.people.get(incoming_msg.personId)
    #firstName = sender.firstName()
    
    file_df = pd.DataFrame()
    file_df['Date'] = [cur_date]
    file_df['Start time'] = [start_time2]
    file_df['End_time'] = [end_time_final]
    
    #start = datetime.time(start_time2)  # Example enter time
    #end = datetime.time(end_time)  # Example start time
    #formatt = '%H%M'
    #time = datetime.strptime(str(start_time2), formatt) - datetime.strptime(str(end_time), formatt)
    
    #file_df["Total time"] = [time]
    #filename = (incoming_msg.text).replace("/save","")
    #filename = (end_time.replace(" ",""))
    timerec_file = openpyxl.Workbook()
    timerec_file.save(os.getcwd() + "\\" + "\'s_time_recording.xlsx")
    writer = pd.ExcelWriter(os.getcwd() + "\\" + "\'s_time_recording.xlsx", engine='openpyxl')
    file_df.to_excel(writer, index=False)
    writer.save()  
    #print("Saved to " + os.getcwd() + "\\" + sender + "'s_time_recording.xlsx" + " successfully.")
    reply = ("Saved to: " + os.getcwd() + "\\" + "\'s_time_recording.xlsx")
    
    
#     abs_path = os.path.abspath(os.getcwd() + "\\" + "\'s_time_recording.xlsx")
#     rooms = api.rooms.list()
#     file_list = [abs_path]
#     api.messages.create(roomId=incoming_msg.id, files=file_list)
    
    response = Response()

    # Set the text of the reply.
    response.text = "Here's a fun little meme."

    # Craft a URL for a file to attach to message
    u = os.getcwd() + "\\" + "\'s_time_recording.xlsx"
    #u = u + "aaaaaalll-righty-then-alrighty-meme.jpg"
    response.files = u
    return response

     #ROOM_ID = incoming_msg.roomId()
#     FILE_PATH = "<the_path_to_the_local_file>"


#     # Create a WebexTeamsAPI connection object; uses your WEBEX_TEAMS_ACCESS_TOKEN
#     api = WebexTeamsAPI()


#     # Let's make sure the file exists
#     if not os.path.isfile(FILE_PATH):
#         print("ERROR: File {} does not exist.".format(FILE_PATH))


#     # Not a requirement but to be completely clear let's make sure we are using
#     # an absolute path.
#     abs_path = os.path.abspath(FILE_PATH)


#     # The files parameter expects to receive a list containing a single string with
#     # the path to the file to be uploaded.
#     file_list = [abs_path]


#     # It takes just a single statement to upload the file and create the message
#     api.messages.create(roomId=ROOM_ID, files=file_list)
        

bot.set_help_message("Welcome to the time recording bot! \n\n I understand the following commands:\n")

# Add new commands to the box.
bot.set_greeting(greeting)
bot.add_command("/recordtime", "Record date, start end time.", date)
bot.add_command("/start", "Record start time.", start_time)
bot.add_command("/end", "Record end time.", end_time)
bot.add_command("/save", "Save recorded data to .xlsx file.", save)
bot.remove_command("/echo")


if __name__ == "__main__":
    # Run Bot
    bot.run(host="127.0.0.1", port="3978")

Teams Bot Email: timerec@webex.bot
Teams Token: REDACTED
Found existing webhook.  Updating it.


 * Serving Flask app "Time Recording Bot" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Configuring Webhook. 
Webhook ID: Y2lzY29zcGFyazovL3VzL1dFQkhPT0svYzBlMjg1YjQtMDUzMi00N2I3LWFlZDMtNTIxYTQ2MDNhMzZk
 * Running on http://127.0.0.1:3978/ (Press CTRL+C to quit)
Message from: elandman@cisco.com
Found command: /recordtime
127.0.0.1 - - [22/Feb/2020 00:04:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2020 00:04:14] "POST / HTTP/1.1" 200 -
Message from: elandman@cisco.com
Found command: /start
127.0.0.1 - - [22/Feb/2020 00:04:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2020 00:04:20] "POST / HTTP/1.1" 200 -
Message from: elandman@cisco.com
Found command: /end
127.0.0.1 - - [22/Feb/2020 00:04:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2020 00:04:26] "POST / HTTP/1.1" 200 -
Message from: elandman@cisco.com
Found command: /save
[2020-02-22 00:04:41,698] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\elandman\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full

'Time Recording Bot'

In [ ]:

# Find all rooms that have 'webexteamssdk Demo' in their title
all_rooms = api.rooms.list()
demo_rooms = [room for room in all_rooms if 'webexteamssdk Demo' in room.title]

# Delete all of the demo rooms
for room in demo_rooms:
    api.rooms.delete(room.id)

# Create a new demo room
demo_room = api.rooms.create('webexteamssdk Demo')

# Add people to the new demo room
email_addresses = ["test01@cmlccie.com", "test02@cmlccie.com"]
for email in email_addresses:
    api.memberships.create(demo_room.id, personEmail=email)

# Post a message to the new room, and upload a file
api.messages.create(demo_room.id, text="Welcome to the room!",
                    files=["https://www.webex.com/content/dam/wbx/us/images/dg-integ/teams_icon.png"])